In [1]:
import psycopg2

In [2]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=alpha")
except psycopg2.Error as e:
    print('Could not make connection to the Postgre database')
    print(e)

In [3]:
try:
    cursor = conn.cursor()
except psycopg2.Error as e:
    print('Could not create a cursor to the database')
    print(e)

In [4]:
conn.set_session(autocommit=True)

<h3>Online Music Store Database</h3>

In [5]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS CUSTOMER_TRANSACTIONS (
            CUSTOMER_ID INT,
            STORE_ID INT,
            SPENT NUMERIC
        );
    ''')
except psycopg2.Error as e:
    print('Could not create trasnactions table')
    print(e)

In [6]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS CUSTOMER (
            CUSTOMER_ID INT,
            NAME VARCHAR,
            REWARDS VARCHAR(1)
        );
    ''')
except psycopg2.Error as e:
    print('Could not create customer table')
    print(e)

In [7]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS STORE (
            STORE_ID INT,
            STATE VARCHAR
        );
    ''')
except psycopg2.Error as e:
    print('Could not create store table')
    print(e)

In [8]:
try:
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ITEMS_PURCHASED (
            CUSTOMER_ID INT,
            ITEM_NUMBER INT,
            ITEM_NAME VARCHAR
        );
    ''')
except psycopg2.Error as e:
    print('Could not create items_purchased table')
    print(e)

Inserts

In [17]:
query = '''
    INSERT INTO CUSTOMER_TRANSACTIONS (
        CUSTOMER_ID,
        STORE_ID,
        SPENT
    ) 
    VALUES (
        %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 1, 20.50))
    cursor.execute(query, (2, 1, 35.21))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [18]:
query = '''
    INSERT INTO CUSTOMER (
        CUSTOMER_ID,
        NAME,
        REWARDS
    ) 
    VALUES (
        %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'Amanda', 'Y'))
    cursor.execute(query, (2, 'Toby', 'N'))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [19]:
query = '''
    INSERT INTO STORE (
        STORE_ID,
        STATE
    ) 
    VALUES (
        %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 'CA'))
    cursor.execute(query, (2, 'WA'))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

In [22]:
query = '''
    INSERT INTO ITEMS_PURCHASED (
        CUSTOMER_ID,
        ITEM_NUMBER,
        ITEM_NAME
    ) 
    VALUES (
        %s, %s, %s
    ) 
'''
try:
    cursor.execute(query, (1, 1, 'Rubber Soul'))
    cursor.execute(query, (2, 3, 'Let It Be'))
except psycopg2.Error as e:
    print('Could not insert into table')
    print(e)

<h4>Find:</h4>
<ul> 
    <li>Customers</li>
    <li>Info About customers</li>
    <li>Info about products</li>
    <li>Rewards status</li> 
</ul>

<h4>For spends more than $30</h4>

In [9]:
query = '''
    SELECT
           A.CUSTOMER_ID
         , B.NAME
         , B.REWARDS
         , C.ITEM_NUMBER
         , C.ITEM_NAME
    FROM 
        CUSTOMER_TRANSACTIONS A 
            LEFT JOIN CUSTOMER B ON A.CUSTOMER_ID = B.CUSTOMER_ID
            LEFT JOIN ITEMS_PURCHASED C ON A.CUSTOMER_ID = C.CUSTOMER_ID
    WHERE
        SPENT >= 30

'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not fetch from tables')
    print(e)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(2, 'Toby', 'N', 3, 'Let It Be')


<h4>Find:</h4>
<ul> 
    <li>How much did store 1 make?</li>
</ul>

In [12]:
query = '''
    SELECT
             A.STORE_ID
           , B.STATE
           , SUM(A.SPENT)
    FROM 
        CUSTOMER_TRANSACTIONS A 
            LEFT JOIN STORE B ON A.STORE_ID = B.STORE_ID
    WHERE A.STORE_ID = 1
    GROUP BY A.STORE_ID, B.STATE

'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not fetch from tables')
    print(e)

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(1, 'CA', Decimal('55.71'))


In [14]:
# Dropping all the tables
query = '''
    DROP TABLE IF EXISTS CUSTOMER_TRANSACTIONS;
    DROP TABLE IF EXISTS STORE;
    DROP TABLE IF EXISTS CUSTOMER;
    DROP TABLE IF EXISTS ITEMS_PURCHASED;
'''
try:
    cursor.execute(query)
except psycopg2.Error as e:
    print('Could not drop table')
    print(e)